## FSDP Tutorial

In this tutorial, we will walk through the implementation of Fully Sharded Data Parallel (FSDP) with Zero2 sharding strategy in `thunder`.

#### Introduction

In recent times, the LLM models have grown so large that all the model parameters don't fit on a single GPU. To circumvent this problem, there are various strategies like Tensor Parallel, Pipeline Parallel, Fully Sharded Data Parallel, etc to train these large models. In this tutorial, we discuss and implement Zero2 strategy for Fully Sharded Data Parallel (FSDP).


#### What is Zero2 strategy for FSDP?

In this strategy, we shard the model parameters across all the availabe GPUs. That is each GPU holds onto only a chunk of the parameter. During the forward pass, all GPUs call `all_gather` communication primitive to gather the parameters from other GPUs. Unlike Zero3 strategy which frees the parameter after forward pass, we save these unsharded parameters for backward pass. This is to save the overhead of extra communication. In the backward pass, we utilize the saved parameters and compute the gradients. Once the gradients are computed, we use `reduce_scatter` communication primitive to reduce (average) the gradients across all GPUs and scatter those gradients so that a given GPU holds only a chunk of gradient.

For more information on FSDP, we recommend reading
1. PyTorch FSDP: Experiences on Scaling Fully Sharded Data Parallel - [Link](https://arxiv.org/abs/2304.11277)
2. ZeRO: Memory Optimizations Toward Training Trillion Parameter Models - [Link](https://arxiv.org/abs/1910.02054)

In [ ]:
import torch
import torch.distributed
import thunder
import thunder.distributed
from IPython.display import Code

For this example we will have a simple model `Linear(Tanh(Linear(x)))` which will be sharded over 2 GPUs

**NOTE**: We are generating the abstract trace so we don't actually need a system with 2 GPUs for this. It is only required when we execute this trace.

In [2]:
device='cuda'
dim = 64
def create_model():
    layers = [torch.nn.Linear(dim, dim, bias=False),
              torch.nn.Tanh(),
              torch.nn.Linear(dim, dim, bias=False)]
    return torch.nn.Sequential(*layers).to(device)

# Model
model = create_model()
# Input
x = torch.randn(dim, dim, device=device)


In [3]:
def wrap_as_highlighted_code(trace):
    return Code(str(trace), language="python")

#### Step 1 : Configuration

For our implementation of FSDP, we will generate the trace where we are sharding our model over 2 GPU

In [4]:
# FSDP Config 
# Usually these values are set in the environment by `torchrun` but for this example
# we will set them ourselves
world_size = 2  # We have two processes.
global_rank = 0  # Current process is the very first process.

#### Step 2: Function to shard parameters

Next step is to write a function which will actually shard the parameters over 0-dim.

In [5]:
# NOTE: We shard over 0th dimension of the param.
def shard_param(param: torch.Tensor, rank: int, world_size: int, name: str) -> None:
    # We will keep it simple and error if param's 0th dim is not divisible by ``world_size``.
    # Alternative is that we can pad our parameters so that they are divisible by `world_size`.
    assert param.shape[0] % world_size == 0,(
        f"Current sharding requires the first dimension of the parameter {name!r} ({param.shape[0]})"
        f" to be divisible by the world size ({world_size})"
    )
    chunk_size = param.shape[0] // world_size

    # rank helps us determine which chunk of the parameter we will hold.
    shard = param.data.narrow(0, chunk_size * rank, chunk_size).clone()
    param.data = shard

# Shard each parameter of the model
for param_name, param in model.named_parameters():
    shard_param(param, global_rank, world_size, param_name)
    # Mark the param to denote that it is sharded.
    # This is required by the synchronization primitive we will use below.
    param.ddp_type = thunder.core.proxies.DDPType.FULLY_SHARDED

In [6]:
# Verify our model looks as expected
model

Sequential(
  (0): Linear(in_features=64, out_features=64, bias=False)
  (1): Tanh()
  (2): Linear(in_features=64, out_features=64, bias=False)
)

In [7]:
# Let us verify that we have actually sharded the parameters.
# Checking if the weight of 1st Linear layer is sharded over 0th dim.
assert model[0].weight.shape == (dim / world_size, dim)

#### Step 3: Add an operation to synchronize the parameters before calling the model.forward.

We have to create a process group. This is needed because the synchronization primitive `synchronize` that we will use to gather and scatter our weights in forward and backward requires a process group.

In [8]:
# Create a process group
options = torch.distributed.distributed_c10d.ProcessGroup.Options(backend="nccl")
process_group = torch.distributed.distributed_c10d.ProcessGroup(torch.distributed.distributed_c10d.Store(),
                                                     global_rank, world_size, options)
torch.distributed.distributed_c10d.GroupMember.WORLD = process_group

In [9]:
# `preprocess` gives us the functional version of the model which
# takes as inputs all the parameters and the expected arguments.
# NOTE: `thunder.common.preprocess` is not meant for general use
#       and used only for brevity of code. It will be updated
#       to a newer mechanism which is meant to be public facing. 
functional_forward = thunder.common.preprocess(model, is_module=True)

# This function creates a model with synchronization
# before calling the forward pass.
def model_with_syncs(*params, x):
    # We call `prims.synchronize` on all the parameters.
    # This is essentially calling `all_gather` so that we have the complete
    # parameter before we actually to the forward computation.
    unsharded_params = []
    for param in params:
        unsharded_params.append(thunder.distributed.prims.synchronize(param, process_group))

    return functional_forward(*unsharded_params, x)

Let us now see what the trace of our model looks like with all the synchronization.

Two main observations regarding the below trace 
1. We can observe the `prims.synchronize` that we inserted using `model_with_syncs`.
2. Output of the `prims.synchronize` have the shape of unsharded (original) parameter.

With this, we have implemented the FSDP for the forward pass of our model.

In [10]:
trace = thunder.trace()(model_with_syncs, *model.parameters(), x=x)

wrap_as_highlighted_code(trace)

# Constructed by Dead Code Elimination (took 0 milliseconds)
import thunder
import thunder.distributed.prims
import thunder.torch as ltorch
import torch
from thunder.executors.torchex import no_autocast

@torch.no_grad()
@no_autocast()
def model_with_syncs(*params, x):
  # params 
  # x 
  t0, \
  t1, \
  = params
  t2 = thunder.distributed.prims.synchronize(t0, _torch_distributed_distributed_c10d_ProcessGroup_0)  # t2
  t3 = thunder.distributed.prims.synchronize(t1, _torch_distributed_distributed_c10d_ProcessGroup_0)  # t3
  t4 = ltorch.linear(x, t2, None)  # t4
    # t4 = prims.linear(x, t2, None)  # t4
  t5 = ltorch.tanh(t4)  # t5
    # t5 = prims.tanh(t4)  # t5
  t6 = ltorch.linear(t5, t3, None)  # t6
    # t6 = prims.linear(t5, t3, None)  # t6
  return t6

For backward, we don't have to do anything because `thunder` already knows how to compute the backward of `prims.synchronize`. We can verify that by using the `value_and_grad` transform to generate the complete forward and backward trace together.

Observations for the trace below:
1. `prims.synchronize` from previous trace is now decomposed into `prims.all_gather` and `prims.wait`. So, we can clearly see that we make a communication call to gather the parameter (which is asynchronous) and wait till we have the complete parameter.
2. At the end of the trace (after the forward and the backward computation), we see calls to `prims.reduce_scatter` and `prims.wait`. This takes care of reducing the gradients across all the GPUs and sharding them. One thing to note, for averaging gradients with low dynamic range dtype like `float16`, if we naively sum the gradients across GPUs before dividing by `world_size`, it can lead to overflows. So we scale the gradient tensor with `world_size`, before calling `reduce_scatter` with `sum` reduction to effectively average the gradients without overflow.

In [11]:
from thunder.core.transforms import value_and_grad

forward_and_backward_model = value_and_grad(model_with_syncs)

forward_backward_trace = thunder.trace()(forward_and_backward_model, *model.parameters(), x=x)

wrap_as_highlighted_code(forward_backward_trace)

# Constructed by Dead Code Elimination (took 0 milliseconds)
import thunder
import thunder.core.devices as devices
import thunder.core.dtypes as dtypes
import thunder.core.prims as prims
import thunder.distributed.prims
import thunder.torch as ltorch
import torch
from thunder.executors.torchex import no_autocast

@torch.no_grad()
@no_autocast()
def _value_and_grad(*args, **kwargs):
  # args 
  # kwargs 
  t0, \
  t1, \
  = args
  t2 = kwargs['x']
  t3 = prims.full((64, 64), 1, device=devices.Device("cuda:0"), dtype=dtypes.float32)  # t3
  p4 = thunder.distributed.prims.all_gather(t0, _torch_distributed_distributed_c10d_ProcessGroup_0, True)  # p4
  t5 = thunder.distributed.prims.wait(p4)  # t5
  p6 = thunder.distributed.prims.all_gather(t1, _torch_distributed_distributed_c10d_ProcessGroup_0, True)  # p6
  t7 = thunder.distributed.prims.wait(p6)  # t7
  t8 = prims.linear(t2, t5, None)  # t8
  t9 = prims.tanh(t8)  # t9
  t10 = prims.linear(t9, t7, None)  # t10
  t11 = ltorch.reshape(t3, -1, 64)  # t11
    # t11 = prims.reshape(t3, (64, 64))  # t11
  t12 = ltorch.matmul(t11, t7)  # t12
    # t12 = prims.matmul(t11, t7)  # t12
  t13 = ltorch.reshape(t3, -1, 64)  # t13
    # t13 = prims.reshape(t3, (64, 64))  # t13
  t14 = prims.transpose(t13, (1, 0))  # t14
  t15 = ltorch.reshape(t9, -1, 64)  # t15
    # t15 = prims.reshape(t9, (64, 64))  # t15
  t16 = ltorch.matmul(t14, t15)  # t16
    # t16 = prims.matmul(t14, t15)  # t16
  t17 = ltorch.mul(t9, t9)  # t17
    # t17 = prims.mul(t9, t9)  # t17
  t18 = ltorch.sub(1.0, t17, alpha=None)  # t18
    # t18 = prims.sub(1.0, t17)  # t18
  t19 = ltorch.mul(t12, t18)  # t19
    # t19 = prims.mul(t12, t18)  # t19
  t20 = ltorch.reshape(t19, -1, 64)  # t20
    # t20 = prims.reshape(t19, (64, 64))  # t20
  t21 = ltorch.matmul(t20, t5)  # t21
    # t21 = prims.matmul(t20, t5)  # t21
  t22 = ltorch.reshape(t19, -1, 64)  # t22
    # t22 = prims.reshape(t19, (64, 64))  # t22
  t23 = prims.transpose(t22, (1, 0))  # t23
  t24 = ltorch.reshape(t2, -1, 64)  # t24
    # t24 = prims.reshape(t2, (64, 64))  # t24
  t25 = ltorch.matmul(t23, t24)  # t25
    # t25 = prims.matmul(t23, t24)  # t25
  t26 = ltorch.true_divide(t16, 2)  # t26
    # _ = prims.convert_element_type(2, float)
    # t26 = prims.div(t16, 2.0)  # t26
  p27 = thunder.distributed.prims.reduce_scatter(t26, _DistributedReduceOps_1, _torch_distributed_distributed_c10d_ProcessGroup_0, True)  # p27
  t28 = thunder.distributed.prims.wait(p27)  # t28
  t29 = ltorch.true_divide(t25, 2)  # t29
    # _ = prims.convert_element_type(2, float)
    # t29 = prims.div(t25, 2.0)  # t29
  p30 = thunder.distributed.prims.reduce_scatter(t29, _DistributedReduceOps_1, _torch_distributed_distributed_c10d_ProcessGroup_0, True)  # p30
  t31 = thunder.distributed.prims.wait(p30)  # t31
  return (t10, (t31, t28, {'x': t21}))

The above trace, only contains primitive which specifies the semantic of an operation abstractly but doesn't perform the actual computation.

Now we will generate the execution trace which can actually perform the compute.

In the execution trace generated below, we can see that all the primitives have been replaced with actually PyTorch operations. Also, our synchronization primitives have been replaced with PyTorch implementation provided by thunder i.e. `torch_all_gather_prim_impl`, `torch_reduce_scatter_prim_impl`, `torch_wait_prim_impl`.

In [12]:
optimized_trace = thunder.transform_for_execution(forward_backward_trace, executors_list=thunder.get_always_executors())

# Grab the final trace
exec_trace = optimized_trace[-1]
wrap_as_highlighted_code(exec_trace)

# Constructed by Delete Last Used (took 0 milliseconds)
import torch
import torch.nn.functional
from thunder.executors.torchex import no_autocast

@torch.no_grad()
@no_autocast()
def _value_and_grad(*args, **kwargs):
  # args 
  # kwargs 
  t0, \
  t1, \
  = args
  del args
  t2 = kwargs['x']
  del kwargs
  t3 = torch.full((64, 64), 1, device=torch.device("cuda:0"), dtype=torch.float32)  # t3
    # t3 = ltorch.full((64, 64), 1, device=torch.device("cuda:0"), dtype=torch.float32)  # t3
      # t3 = prims.full((64, 64), 1, device=devices.Device("cuda:0"), dtype=dtypes.float32)  # t3
  p4 = torch_all_gather_prim_impl(t0, _torch_distributed_distributed_c10d_ProcessGroup_2, True)  # p4
  del t0
  t5 = torch_wait_prim_impl(p4)  # t5
  del p4
  p6 = torch_all_gather_prim_impl(t1, _torch_distributed_distributed_c10d_ProcessGroup_2, True)  # p6
  del t1
  t7 = torch_wait_prim_impl(p6)  # t7
  del p6
  t8 = torch.nn.functional.linear(t2, t5, None)  # t8
    # t8 = ltorch.linear(t2, t5, None)  # t8
      # t8 = prims.linear(t2, t5, None)  # t8
  t9 = torch.tanh(t8)  # t9
    # t9 = ltorch.tanh(t8)  # t9
      # t9 = prims.tanh(t8)  # t9
  del t8
  t10 = torch.nn.functional.linear(t9, t7, None)  # t10
    # t10 = ltorch.linear(t9, t7, None)  # t10
      # t10 = prims.linear(t9, t7, None)  # t10
  t11 = torch.reshape(t3, (-1, 64))  # t11
    # t11 = ltorch.reshape(t3, (-1, 64))  # t11
      # t11 = prims.reshape(t3, (64, 64))  # t11
  t12 = torch.matmul(t11, t7)  # t12
    # t12 = ltorch.matmul(t11, t7)  # t12
      # t12 = prims.matmul(t11, t7)  # t12
  del t11, t7
  t13 = torch.reshape(t3, (-1, 64))  # t13
    # t13 = ltorch.reshape(t3, (-1, 64))  # t13
      # t13 = prims.reshape(t3, (64, 64))  # t13
  del t3
  t14 = torch.permute(t13, (1, 0))  # t14
    # t14 = ltorch.permute(t13, (1, 0))  # t14
      # t14 = prims.transpose(t13, (1, 0))  # t14
  del t13
  t15 = torch.reshape(t9, (-1, 64))  # t15
    # t15 = ltorch.reshape(t9, (-1, 64))  # t15
      # t15 = prims.reshape(t9, (64, 64))  # t15
  t16 = torch.matmul(t14, t15)  # t16
    # t16 = ltorch.matmul(t14, t15)  # t16
      # t16 = prims.matmul(t14, t15)  # t16
  del t14, t15
  t17 = torch.mul(t9, t9)  # t17
    # t17 = ltorch.mul(t9, t9)  # t17
      # t17 = prims.mul(t9, t9)  # t17
  del t9
  t18 = torch.sub(1.0, t17)  # t18
    # t18 = ltorch.sub(1.0, t17, alpha=None)  # t18
      # t18 = prims.sub(1.0, t17)  # t18
  del t17
  t19 = torch.mul(t12, t18)  # t19
    # t19 = ltorch.mul(t12, t18)  # t19
      # t19 = prims.mul(t12, t18)  # t19
  del t12, t18
  t20 = torch.reshape(t19, (-1, 64))  # t20
    # t20 = ltorch.reshape(t19, (-1, 64))  # t20
      # t20 = prims.reshape(t19, (64, 64))  # t20
  t21 = torch.matmul(t20, t5)  # t21
    # t21 = ltorch.matmul(t20, t5)  # t21
      # t21 = prims.matmul(t20, t5)  # t21
  del t20, t5
  t22 = torch.reshape(t19, (-1, 64))  # t22
    # t22 = ltorch.reshape(t19, (-1, 64))  # t22
      # t22 = prims.reshape(t19, (64, 64))  # t22
  del t19
  t23 = torch.permute(t22, (1, 0))  # t23
    # t23 = ltorch.permute(t22, (1, 0))  # t23
      # t23 = prims.transpose(t22, (1, 0))  # t23
  del t22
  t24 = torch.reshape(t2, (-1, 64))  # t24
    # t24 = ltorch.reshape(t2, (-1, 64))  # t24
      # t24 = prims.reshape(t2, (64, 64))  # t24
  del t2
  t25 = torch.matmul(t23, t24)  # t25
    # t25 = ltorch.matmul(t23, t24)  # t25
      # t25 = prims.matmul(t23, t24)  # t25
  del t23, t24
  t26 = torch.true_divide(t16, 2)  # t26
    # t26 = ltorch.true_divide(t16, 2)  # t26
      # _ = prims.convert_element_type(2, float)
      # t26 = prims.div(t16, 2.0)  # t26
  del t16
  p27 = torch_reduce_scatter_prim_impl(t26, _DistributedReduceOps_3, _torch_distributed_distributed_c10d_ProcessGroup_2, True)  # p27
  del t26
  t28 = torch_wait_prim_impl(p27)  # t28
  del p27
  t29 = torch.true_divide(t25, 2)  # t29
    # t29 = ltorch.true_divide(t25, 2)  # t29
      # _ = prims.convert_element_type(2, float)
      # t29 = prims.div(t25, 2.0)  # t29
  del t25
  p30 = torch_r

#### Step 4 : Running the actual computation

Running the actual computation will require setting up 2 processes and running our above code in both those processes (which can be tricky with Jupyter Notebook). Instead, we will write a small script and run it with `torchrun` which takes care of setting up the processes and relevant state.

**NOTE**: This requires device running this notebook to have at least 2-GPUs

In the example below, we will use `thunder.distributed.fsdp` which does the same as what we did above (with some extra checks). The code below should look familiar as it is roughly all the above pieces in a single script. 

In [13]:
%%writefile thunder_fsdp_simple_example.py

# imports
from thunder.tests.lit_gpt_model import GPT, Config
import torch
import torch.distributed
import thunder
import thunder.distributed
import os

# # # # # # # #
# Create Model
# # # # # # # #

# NOTE: We create the model on CPU.
device='cpu'
dim = 64
def create_model():
    layers = []
    layers.append(torch.nn.Linear(dim, dim))
    layers.append(torch.nn.ReLU())
    layers.append(torch.nn.Linear(dim, dim))
    return torch.nn.Sequential(*layers).to(device)

# Model
model = create_model()
# Input
x = torch.randn(dim, dim, device=device)

# # # # # # # #
# Setup for distributed
# # # # # # # #
torch.distributed.init_process_group(backend='nccl')

rank = int(os.environ["LOCAL_RANK"])

device = f"cuda:{rank}"

# # # # # # # #
# Move inputs to correct device
# # # # # # # #
x = x.to(device)

# # # # # # # #
# Wrap the model in thunder.distributed.fsdp
# # # # # # # #

# thunder.distributed.fsdp takes care of moving the parameter
# shard to the correct GPU for the current process.
cmodel = thunder.jit(thunder.distributed.fsdp(model))

# Run the forward pass.
cmodel(x)

# # # # # # # #
# Check the traces
# # # # # # # #
fwd_traces, bwd_traces = thunder.last_traces(cmodel)

# # # # # # # #
# Print and check to see if they match ours
# # # # # # # #
if rank == 0:
    print(fwd_traces[-1])
    print("*******"* 8)
    print(bwd_traces[-1])

Overwriting thunder_fsdp_simple_example.py


Let us run the above script and check what the trace looks like.

We can observe that forward trace has `torch_all_gather_prim_impl` to gather the parameter before forward pass and the backward trace has `torch_reduce_scatter_prim_impl` to reduce and scatter the gradients back to different GPUs. This is similar to our implementation above.

In [14]:
!torchrun --nproc_per_node=2 thunder_fsdp_simple_example.py

[2024-03-06 15:59:54,829] torch.distributed.run: [WARNING] 
[2024-03-06 15:59:54,829] torch.distributed.run: [WARNING] *****************************************
[2024-03-06 15:59:54,829] torch.distributed.run: [WARNING] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
[2024-03-06 15:59:54,829] torch.distributed.run: [WARNING] *****************************************
/home/kkalambarkar/miniconda3/envs/pytorch-dev/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/kkalambarkar/miniconda3/envs/pytorch-dev/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please 

#### Conclusion

We have created our implementation of FSDP to shard our model across multiple GPUs. In the process, we also learned that 
1. `thunder` provides us with primitives for synchronization across mutiple GPUs.
2. `thunder` also takes care of implementing the backward support, so we don't have to explicitly do anything to get the backward working.
3. We can just easily apply `thunder.distributed.fsdp` to our model and it will take care of sharding the parameters and also adding synchronizations to our model. Also, we can easily check the modifications by inspecting the traces.